In [1]:
!pip install torchvision transformers torch accelerate diffusers
!pip install xformers gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 59.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
!pip install --upgrade --force-reinstall torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 54.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.1/204.1 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
!pip install gradio

In [2]:
!pip install gtts

In [3]:
import torch
import gradio as gr
import numpy as np
from PIL import Image, ImageEnhance
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from diffusers import StableDiffusionPipeline
from gtts import gTTS
import os
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cuda.matmul.allow_tf32 = True

model_name = "microsoft/phi-2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name, torch_dtype=torch.float16, device_map="auto"
)
comic_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

model_id = "stabilityai/sd-turbo"
pipe = StableDiffusionPipeline.from_pretrained(
    model_id, torch_dtype=torch.float16
).to("cuda")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

...

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.46G [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


In [7]:
art_styles = {
    "Biology Diagram": (
        "Highly detailed, scientifically accurate biological illustrations with precisely labeled structures. "
        "Realistic textures for tissues, organs, and cells, ensuring anatomical accuracy. "
        "Cross-sectional views and zoomed-in details for better understanding. Styled like medical and biology textbooks."
    ),
    "Physics Visualization": (
        "Scientifically precise depictions of physics phenomena, including accurate vector representations, motion trails, "
        "and force diagrams. High-contrast color schemes to enhance clarity, with properly labeled components. "
        "3D-rendered visualizations of physical interactions for improved comprehension. Inspired by academic physics models."
    ),
    "Astronomy": (
        "Deep space imagery with ultra-realistic planet and star textures, ensuring scientific accuracy. "
        "NASA-style visualization with correct lighting, atmospheric effects, and astrophysical details. "
        "Includes celestial objects such as nebulae, galaxies, and planetary rings, designed for educational use."
    ),
    "Microscopic View": (
        "Electron microscope-style rendering with extreme close-ups of cell structures, bacteria, viruses, and microscopic organisms. "
        "High-magnification accuracy with realistic textures, depth of field, and appropriate contrast. "
        "True-to-life color grading or grayscale imaging based on microscopy techniques. Dark or black backgrounds for clarity."
    ),
    "Chemistry Visualization": (
        "Accurate molecular models, reaction mechanisms, and atomic structures rendered with scientific precision. "
        "Detailed depictions of chemical bonds, electron orbitals, and reaction dynamics with appropriate labeling. "
        "Includes organic and inorganic chemistry illustrations, periodic table-based color coding, and thermodynamic properties visualization."
    )
}

def generate_comic_story(topic, num_panels):
    """Generates a scientific comic strip storyline"""
    instruction = (
        f"Create a {num_panels}-panel scientific comic strip about {topic}. "
        "Each panel should provide accurate explanations with clear scientific concepts, minimal exaggeration, and labeled diagrams.\n\n"
        "Scientific Comic Strip Panels:\n"
    )

    response = comic_pipeline(
        instruction, max_new_tokens=400, temperature=0.5, repetition_penalty=1.2, do_sample=True
    )[0]['generated_text']

    comic_breakdown = response.replace(instruction, "").strip()
    comic_panels = [line.strip() for line in comic_breakdown.split("\n") if line.strip()][:num_panels]

    return "\n".join(comic_panels)

def refine_prompt(description, style):
    """Refines panel descriptions into structured, detailed prompts for better image generation"""
    return (
        f"{description}. The scene is visually dynamic with a strong focus on scientific accuracy and clarity. "
        f"Diagrams and illustrations are labeled and detailed, adding depth and realism. "
        f"Lighting and shading enhance the scientific visualization. {art_styles.get(style, 'Scientific illustration')}."
    )

def enhance_image(image):
    """Enhances sharpness and contrast for clearer comic panels"""
    enhancer = ImageEnhance.Sharpness(image)
    image = enhancer.enhance(2.0)

    enhancer = ImageEnhance.Contrast(image)
    image = enhancer.enhance(1.3)

    return image

def generate_comic_images(edited_prompts, style, direction="LTR"):
    """Generates comic images from user-edited panel descriptions with white gaps between images."""
    panels = edited_prompts.strip().split("\n")

    comic_images = [generate_comic_image(p, style) for p in panels]

    if not comic_images:
        return "No images were generated.", None
    if direction == "RTL":
        comic_images = comic_images[::-1]

    pages = []
    panel_size = 512  # Each panel's size
    gap = 20  # White space between panels
    cols = 3
    rows = (len(comic_images) + cols - 1) // cols
    page_width = (panel_size + gap) * cols + gap
    page_height = (panel_size + gap) * rows + gap

    for i in range(0, len(comic_images), 9):
        page_panels = comic_images[i:i+9]
        page = Image.new("RGB", (page_width, page_height), "white")

        for j, img in enumerate(page_panels):
            if img:
                x_offset = (j % cols) * (panel_size + gap) + gap
                y_offset = (j // cols) * (panel_size + gap) + gap
                page.paste(img, (x_offset, y_offset))

        pages.append(page)

    return edited_prompts, pages


def generate_comic_image(description, style):
    """Generates a single scientific illustration panel"""
    prompt = f"{description}, {art_styles.get(style, 'Scientific illustration')}, accurate scientific representation, minimal distortion"
    negative_prompt = "cartoonish, exaggerated, distorted, low accuracy, fantasy, text, watermark"

    try:
        image = pipe(
            prompt,
            negative_prompt=negative_prompt,
            width=512, height=512,
            num_inference_steps=30,
            guidance_scale=7
        ).images[0]
        return image
    except Exception as e:
        print(f"Error generating image: {e}")
        return None

def text_to_speech(text):
    """Convert generated scientific story to speech."""
    tts = gTTS(text, lang="en")
    audio_path = "scientific_comic_audio.mp3"
    tts.save(audio_path)
    return audio_path
with gr.Blocks() as interface:
    gr.Markdown("# 🧪 AI Scientific Comic Strip Generator")
    gr.Markdown("Generate educational scientific comics with AI-powered text, image, and audio narration.")

    with gr.Row():
        topic_input = gr.Textbox(label="Enter a scientific topic:", value="Photosynthesis")
        num_panels_input = gr.Radio([3, 6, 9], label="Number of Panels", value=3)
        direction_input = gr.Radio(["Left to Right (LTR)", "Right to Left (RTL)"], label="Comic Direction", value="Left to Right (LTR)")

    art_style_input = gr.Radio(list(art_styles.keys()), label="Choose a Scientific Illustration Style")

    generate_story_btn = gr.Button("Generate Story & Images")
    regenerate_images_btn = gr.Button("Regenerate Images")
    generate_audio_btn = gr.Button("Generate Audio Narration")

    generated_story = gr.Textbox(label="Generated Story (Edit if needed)", lines=10)
    comic_strip_output = gr.Gallery(label="Generated Comic Pages")
    audio_output = gr.Audio(label="Narrated Story", autoplay=True)

    def generate_and_display_story(topic, num_panels, direction, style):
        story = generate_comic_story(topic, num_panels)
        direction = "LTR" if direction == "Left to Right (LTR)" else "RTL"
        _, comic_pages = generate_comic_images(story, style, direction)
        return story, comic_pages

    def regenerate_images(edited_story, direction, style):
        direction = "LTR" if direction == "Left to Right (LTR)" else "RTL"
        return generate_comic_images(edited_story, style, direction)

    def generate_audio(story_text):
        return text_to_speech(story_text)

    generate_story_btn.click(generate_and_display_story,
                            inputs=[topic_input, num_panels_input, direction_input, art_style_input],
                            outputs=[generated_story, comic_strip_output])

    regenerate_images_btn.click(regenerate_images, inputs=[generated_story, direction_input, art_style_input], outputs=[generated_story, comic_strip_output])

    generate_audio_btn.click(generate_audio, inputs=[generated_story], outputs=[audio_output])

In [8]:
interface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://78d3aba4c13506674e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
torch.cuda.empty_cache()